In [ ]:
%pip install langchain qdrant_client

In [ ]:
%pip install sentence_transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import qdrant_client
import os
import json
from langchain.text_splitter import CharacterTextSplitter
import uuid
from qdrant_client import models
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
# Set your Qdrant host and API key
os.environ['QDRANT_HOST'] = "https://4483e22b-5513-4e59-a592-6049d9a98352.eu-central-1-0.aws.cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] = "kT3Yv6siCS5-ZESLEPpxUtz87-uAjhv8unoxvanOt_sCXnnvtm0gWA"

# Create Qdrant client
client = qdrant_client.QdrantClient(
    os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [ ]:
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# model = SentenceTransformer('thenlper/gte-base')
# model = SentenceTransformer('thenlper/gte-large')
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
collection_name = "laws_ca_qb"

client.recreate_collection(
	collection_name=collection_name,
	vectors_config=models.VectorParams(
		size=model.get_sentence_embedding_dimension(),
		distance=models.Distance.COSINE
	)
)

True

In [ ]:
import json

# Load your JSON data
with open("/content/drive/My Drive/règlements_codifiés_ca.json") as f:
    data = json.load(f)

# Calculate the length of each piece
total_length = len(data)
pieces = 20
piece_length = total_length // pieces

# Split the data into 20 equal pieces
piece1 = data[:piece_length]
piece2 = data[piece_length:2 * piece_length]
piece3 = data[2 * piece_length:3 * piece_length]
piece4 = data[3 * piece_length:4 * piece_length]
piece5 = data[4 * piece_length:5 * piece_length]
piece6 = data[5 * piece_length:6 * piece_length]
piece7 = data[6 * piece_length:7 * piece_length]
piece8 = data[7 * piece_length:8 * piece_length]
piece9 = data[8 * piece_length:9 * piece_length]
piece10 = data[9 * piece_length:10 * piece_length]
piece11 = data[10 * piece_length:11 * piece_length]
piece12 = data[11 * piece_length:12 * piece_length]
piece13 = data[12 * piece_length:13 * piece_length]
piece14 = data[13 * piece_length:14 * piece_length]
piece15 = data[14 * piece_length:15 * piece_length]
piece16 = data[15 * piece_length:16 * piece_length]
piece17 = data[16 * piece_length:17 * piece_length]
piece18 = data[17 * piece_length:18 * piece_length]
piece19 = data[18 * piece_length:19 * piece_length]
piece20 = data[19 * piece_length:]



In [ ]:
def get_chunks(text):
    # Replace occurrences of "\r\n \r\n" with "\r\n" to handle both cases
    text = text.replace("\r\n \r\n", "\r\n")

    # Split the text into paragraphs using "\r\n" sequence as separator
    paragraphs = text.split("\r\n")

    # Initialize variables to keep track of chunk boundaries
    chunk_size = 0
    chunk_overlap = 200
    current_chunk = ""
    chunks = []

    # Iterate through paragraphs
    for paragraph in paragraphs:
        # Check if adding the current paragraph to the current chunk exceeds the chunk size
        if chunk_size + len(paragraph) > 1000:
            # Add the current chunk to the list of chunks
            chunks.append(current_chunk)

            # Reset current chunk and update chunk size
            current_chunk = paragraph + "\r\n"
            chunk_size = len(paragraph) + 2
        else:
            # Add the current paragraph to the current chunk
            current_chunk += paragraph + "\r\n"
            chunk_size += len(paragraph) + 2

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [ ]:
import concurrent.futures
import json
import os
import uuid
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models


def process_document_chunk(doc_id, source, url, chunk_id, chunk_content):
    # Encode the chunk content using the SentenceTransformer model
    embedding = model.encode([chunk_content])[0]

    # Generate a UUID for the chunk
    chunk_uuid = str(uuid.uuid4())

    # Create a Record object for the chunk
    chunk_record = qdrant_client.http.models.Record(
        id=chunk_uuid,
        vector=embedding.tolist(),
        payload={
            "document_id": doc_id,
            "source": source,
            "url": url,
            "chunk_id": chunk_id,
            "content": chunk_content
        }
    )

    return chunk_record

def upload_chunk_records(records):
    client.upload_records(
        collection_name=collection_name,
        records=records
    )

# Process and upload chunk records for each document
with concurrent.futures.ThreadPoolExecutor() as executor:
    for idx, doc in enumerate(piece20):
        doc_id = doc["id_lr"]
        source = doc["origin"]
        url = doc["url"]
        doc_content = doc["content"]

        # Split the content into chunks
        content_chunks = get_chunks(doc_content)

        # Process and upload chunk records
        chunk_records = []
        for chunk_id, chunk_content in enumerate(content_chunks):
            chunk_record = process_document_chunk(doc_id, source, url, chunk_id, chunk_content)
            chunk_records.append(chunk_record)

            if len(chunk_records) >= 100:  # Adjust batch size as needed
                executor.submit(upload_chunk_records, chunk_records)
                chunk_records = []

        # Upload remaining chunk records
        if chunk_records:
            executor.submit(upload_chunk_records, chunk_records)


In [ ]:
# Choose a sample query in French
query = "De combien de membres se composera l'assemblée législative du Québec ?"

# Define a function to perform a search and print the results
def perform_search_and_print_results(collection_name, query, limit=3):
    search_results = client.search(
        collection_name=collection_name,
        query_vector=model.encode(query).tolist(),
        limit=limit
    )
    print(f"Search results for collection: {collection_name}")
    for result in search_results:
        print("Score:", result.score)
        print("Chunk ID:", result.payload["chunk_id"])
        print("Content:", result.payload["content"])
        print("Document ID:", result.payload["document_id"])
        print()

perform_search_and_print_results(collection_name, query)



Search results for collection: laws_ca_qb
Score: 0.85272074
Chunk ID: 37
Content:  Note marginale : Constitution du conseil législatif 72  Le conseil législatif de Québec se composera de vingt-quatre membres, qui seront nommés par le lieutenant-gouverneur au nom de la Reine, par instrument sous le grand sceau de Québec, et devront, chacun, représenter l’un des vingt-quatre collèges électoraux du Bas-Canada mentionnés à la présente loi; ils seront nommés à vie, à moins que la législature de Québec n’en ordonne autrement sous l’autorité de la présente loi. 
 Note marginale : Qualités exigées des conseillers législatifs 73  Les qualifications des conseillers législatifs de Québec seront les mêmes que celles des sénateurs pour Québec. 
 Note marginale : Cas dans lesquels les sièges des conseillers législatifs deviennent vacants 74  La charge de conseiller législatif de Québec deviendra vacante dans les cas,  mutatis mutandis , où celle de sénateur peut le devenir. 

Document ID: LOI CONSTI